In [ ]:
import pandas as pd
#import italy covid cases
df_covid_iqa_mi = pd.read_csv('covid_iqa_mi.csv',delimiter=',', keep_default_na=False, na_values=[''])
df_covid_iqa_mi['daily_cases'] = df_covid_iqa_mi['daily_cases'].fillna(0)
df_covid_iqa_mi.head()
#print

In [ ]:
# select city
df_covid_iqa_mi = df_covid_iqa_mi.loc[df_covid_iqa_mi['denominazione_provincia'].isin(['Milano'])]
df_covid_iqa_mi[['data','daily_cases','totale_casi']].head()


In [ ]:
# statistiche descrittive qualità aria
y= df_covid_iqa_mi["IQA"]
stats = y.describe()
stats.loc['var'] = y.var()
stats.loc['skew'] = y.skew()
stats.loc['kurt'] = y.kurtosis()
print("statistiche descrittive qualita dell'aria")
print(stats)
# statistiche descrittive delta giornalieri

In [ ]:
# Normal distribution test
import scipy
from scipy import stats
import matplotlib.pyplot as plt

print("Test normality for covid cases")
normality_test(df_covid_iqa_mi['daily_cases'],"daily_cases")
print("Test normality for IQA")
normality_test(df_covid_iqa_mi['IQA'],"IQA")
print("Test normality for MoveIndex")
normality_test(df_covid_iqa_mi['MoveIndex'],"MoveIndex")
#box-cox transformation covid daily
#y_bc_covid = box_cox_transformation(df_covid_iqa['daily_cases'],"daily_cases")


In [ ]:
def normality_test(data,target):
    stat,p = stats.normaltest(data)
    print("Statistics = %.3f, p=%.3f" % (stat,p))
    alpha = 0.05
    if p> alpha:
        print('Data looks Gaussian (fail to reject null hypothesis)')
    else:
        print('Data looks non-Gaussian (reject null hypothesis)')

    #test normalità 
    jarque_bera_test = stats.jarque_bera(data)
    jarque_bera_test

    plt.figure(figsize=(14,6))
    plt.subplot(1,2,1)
    data.hist(bins=50)
    plt.title(target)
    plt.subplot(1,2,2)
    stats.probplot(data, plot=plt);
    data.describe().T

In [ ]:
#box cox transformation
import pmdarima as pm
from pmdarima import preprocessing
def box_cox_transformation(data,target):
    y = data.values
    # Box Cox Transformation per renderla normale
    m_bce = preprocessing.BoxCoxEndogTransformer().fit(y)
    y_bc, _ = m_bce.transform(y)
    
#     plt.figure(figsize=(14,6))
#     plt.subplot(1,2,1)
#     data.hist(bins=50)
#     plt.title(target)
#     plt.subplot(1,2,2)
#     stats.probplot(data, plot=plt);
#     data.describe().T
    return y_bc

In [ ]:
# Pearson correlation
import seaborn as sns; sns.set()
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# minmax scalar
# scaler = MinMaxScaler()
# df_covid_iqa_mi['IQA_tr'] = scaler.fit_transform(df_covid_iqa[['IQA']])
# df_covid_iqa_mi['IQA_tr'] = 1- df_covid_iqa_mi['IQA_tr'] 

# correlation matrix
corr = df_covid_iqa_mi[["daily_cases","IQA","MoveIndex"]].apply(np.log).corr(method='pearson')
print(corr)
sns.heatmap(corr, cmap="BuGn")

In [ ]:
# Let's transform the series
y_tr_cases = transform_function(df_covid_iqa_mi['daily_cases'])
y_tr_iqa = transform_function(df_covid_iqa_mi['IQA'])
y_tr_mi = transform_function(df_covid_iqa_mi['MoveIndex'])

In [ ]:
import numpy as np
def transform_function(df):
    #df = df.apply(np.log).diff().dropna()
    df = df.diff().dropna()
    return df

In [ ]:
print("stationarity test for cases")
kpss_test(y_tr_cases)
print("stationarity test for iqa")
kpss_test(y_tr_iqa)
print("stationarity test for move index")
kpss_test(y_tr_mi)

In [ ]:
# KPSS test
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')


In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
#granger causality test
columns=["daily_cases","IQA","MoveIndex"]
dataset = pd.DataFrame(columns=columns)
dataset["daily_cases"] = y_tr_cases
dataset["IQA"] = y_tr_iqa
dataset["MoveIndex"] = y_tr_mi
print(dataset.head())
print(dataset.shape)
data = grangers_causality_matrix(dataset, variables = dataset.columns,verbose=True)  
#data = 1 - data
print(data)
print(data.shape)
sns.heatmap(data, cmap="BuGn")

In [ ]:
#granger causality test function
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np

def grangers_causality_matrix(data, variables, test = 'ssr_chi2test', maxlag=44, verbose=False):
    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in dataset.columns:
        for r in dataset.index:
            test_result = grangercausalitytests(data[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            dataset.loc[r,c] = min_p_value
    dataset.columns = [var + '_x' for var in variables]
    dataset.index = [var + '_y' for var in variables]

    return dataset